## <span style="color:purple"> Experimental: Pre-annotation of PropBank semantic roles </span>

PropBankPreannotator provides a lexicon-based tagging of PropBank semantic roles [(Palmer et al 2005)](https://www.cs.rochester.edu/~gildea/palmer-propbank-cl.pdf) in text. 
The lexicon has been automatically derived from manually crafted resources of the project EKTB75. 

Before using PropBankPreannotator, you'll need to download the lexicon, as it is not distributed with the EstNLTK package.

* If you create a new instance of PropBankPreannotator and the lexicon is missing, you'll be prompted with a question asking for permission to download the lexicon;
* Alternatively, you can pre-download the lexicon manually via `download` function:

```python
from estnltk import download
download("propbankpreannotator")
```

In order to use PropBankPreannotator, text must be first syntactically annotated. 
Use [StanzaSyntaxTagger](../C_syntax/03_syntactic_analysis_with_stanza.ipynb) from `estnltk_neural` to provide syntactic annotations:

In [1]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
# Initialize Stanza-based syntactic parser
from estnltk_neural.taggers import StanzaSyntaxTagger
syntax_tagger = StanzaSyntaxTagger(input_type='morph_extended', input_morph_layer='morph_extended')

In [2]:
# Initialize PropBankPreannotator working on 'stanza_syntax' layer
from estnltk.taggers.miscellaneous.propbank.preannotator import PropBankPreannotator
propbank_annotator = PropBankPreannotator(input_syntax_layer='stanza_syntax')

In [3]:
# Create example input Text
from estnltk import Text
text = Text('''Ants rääkis asjast Arvole. Seega, ta edastas sõnumi edukalt adressaadile. Küsisin IT-juhilt abi. ''')
# Tag necessary layers
text.tag_layer(['sentences', 'morph_extended'])
syntax_tagger.tag(text)
# Tag semantic role pre-annotations
propbank_annotator.tag(text)

Text(text='Ants rääkis asjast Arvole. Seega, ta edastas sõnumi edukalt adressaadile. Küsisin IT-juhilt abi. ')

In [4]:
# Examine results
text['pre_semantic_roles']

RelationLayer(name='pre_semantic_roles', span_names=('verb', 'arg0', 'arg1', 'arg2', 'arg3', 'arg4', 'arg5', 'argm_mnr', 'argm_tmp', 'argm_loc'), attributes=('sense_id',), relations=[Relation([NamedSpan(verb: ['rääkis']), NamedSpan(arg0: ['Ants']), NamedSpan(arg2: ['Arvole']), NamedSpan(arg3: ['asjast'])], [{'sense_id': 'rääkima_1'}]), Relation([NamedSpan(verb: ['edastas']), NamedSpan(arg0: ['ta']), NamedSpan(arg2: ['adressaadile'])], [{'sense_id': 'edastama_1'}]), Relation([NamedSpan(verb: ['Küsisin']), NamedSpan(arg1: ['abi']), NamedSpan(arg2: ['IT-juhilt'])], [{'sense_id': 'küsima_1'}])])

In [5]:
# Visualize results
text['pre_semantic_roles'].display()

Ants arg0(0) rääkis verb(0) asjast arg3(0) Arvole arg2(0) . Seega, ta arg0(1) edastas verb(1) sõnumi edukalt adressaadile arg2(1) . Küsisin verb(2) IT-juhilt arg2(2) abi arg1(2) .

#### Tagger's parameters

PropBankPreannotator allows to modify the following parameters via constructor:

* `output_flat_layer` -- Whether the output layer will be formatted as a flat layer (that is: a layer without any dependencies). By default, the output layer is not a flat layer, but an enveloping layer around the input syntax layer, because lexicon builds upon nodes of the syntax layer. (Default: False)
* `discard_frames_wo_args` -- Whether frames without any arguments will be included in the output (Default: True).
* `add_verb_class` -- Whether the output layer has extra attribute `verb_class` conveying the verb class from the lexicon (Default: False).
* `add_arg_descriptions` -- Whether the output layer contains extra attributes with argument descriptions, named as `arg0_desc`, `arg1_desc`, ... , `arg5_desc`. (Default: False)
* `add_arg_feats` -- Whether information about arguments' morph/syntactic features will be added to the output layer, named as  `arg0_feats`, `arg1_feats`, ... , `argm_loc_feats`. These are the features that were used in the frame extraction. (Default: False)
* `skip_compound_prt` -- Whether compound verbs will be discarded from frame detection (that is: verbs that have a child with `compound:prt` deprel will be discarded) (Default: True)

Example: set `add_arg_descriptions` to reveal manually provided argument descriptions (if any):

In [6]:
propbank_annotator = PropBankPreannotator(input_syntax_layer='stanza_syntax', add_arg_descriptions=True)
# Remove old layer
text.pop_layer(propbank_annotator.output_layer)
# Add layer with new settings
propbank_annotator.tag(text)
# Examine results
text['pre_semantic_roles']

RelationLayer(name='pre_semantic_roles', span_names=('verb', 'arg0', 'arg1', 'arg2', 'arg3', 'arg4', 'arg5', 'argm_mnr', 'argm_tmp', 'argm_loc'), attributes=('sense_id', 'arg0_desc', 'arg1_desc', 'arg2_desc', 'arg3_desc', 'arg4_desc', 'arg5_desc'), relations=[Relation([NamedSpan(verb: ['rääkis']), NamedSpan(arg0: ['Ants']), NamedSpan(arg2: ['Arvole']), NamedSpan(arg3: ['asjast'])], [{'sense_id': 'rääkima_1', 'arg0_desc': 'rääkija', 'arg1_desc': None, 'arg2_desc': 'adressaat: kellele/kellega', 'arg3_desc': 'millegi kohta/millest', 'arg4_desc': None, 'arg5_desc': None}]), Relation([NamedSpan(verb: ['edastas']), NamedSpan(arg0: ['ta']), NamedSpan(arg2: ['adressaadile'])], [{'sense_id': 'edastama_1', 'arg0_desc': 'edastaja', 'arg1_desc': None, 'arg2_desc': 'adressaat', 'arg3_desc': None, 'arg4_desc': None, 'arg5_desc': None}]), Relation([NamedSpan(verb: ['Küsisin']), NamedSpan(arg1: ['abi']), NamedSpan(arg2: ['IT-juhilt'])], [{'sense_id': 'küsima_1', 'arg0_desc': None, 'arg1_desc': 'mida/et', 'arg2_desc': 'kelleltki/kellegi käest', 'arg3_desc': None, 'arg4_desc': None, 'arg5_desc': None}])])

Example: set `add_arg_feats` to reveal morphosyntactic features triggering each argument:

In [7]:
propbank_annotator = PropBankPreannotator(input_syntax_layer='stanza_syntax', add_arg_feats=True)
# Remove old layer
text.pop_layer(propbank_annotator.output_layer)
# Add layer with new settings
propbank_annotator.tag(text)
# Examine results
text['pre_semantic_roles']

RelationLayer(name='pre_semantic_roles', span_names=('verb', 'arg0', 'arg1', 'arg2', 'arg3', 'arg4', 'arg5', 'argm_mnr', 'argm_tmp', 'argm_loc'), attributes=('sense_id', 'arg0_feats', 'arg1_feats', 'arg2_feats', 'arg3_feats', 'arg4_feats', 'arg5_feats', 'argm_mnr_feats', 'argm_tmp_feats', 'argm_loc_feats'), relations=[Relation([NamedSpan(verb: ['rääkis']), NamedSpan(arg0: ['Ants']), NamedSpan(arg2: ['Arvole']), NamedSpan(arg3: ['asjast'])], [{'sense_id': 'rääkima_1', 'arg0_feats': ['deprel=nsubj'], 'arg1_feats': None, 'arg2_feats': ['deprel=obl', 'case=All'], 'arg3_feats': ['deprel=obl', 'case=Ela'], 'arg4_feats': None, 'arg5_feats': None, 'argm_mnr_feats': None, 'argm_tmp_feats': None, 'argm_loc_feats': None}]), Relation([NamedSpan(verb: ['edastas']), NamedSpan(arg0: ['ta']), NamedSpan(arg2: ['adressaadile'])], [{'sense_id': 'edastama_1', 'arg0_feats': ['deprel=nsubj'], 'arg1_feats': None, 'arg2_feats': ['deprel=obl', 'case=All'], 'arg3_feats': None, 'arg4_feats': None, 'arg5_feats': None, 'argm_mnr_feats': None, 'argm_tmp_feats': None, 'argm_loc_feats': None}]), Relation([NamedSpan(verb: ['Küsisin']), NamedSpan(arg1: ['abi']), NamedSpan(arg2: ['IT-juhilt'])], [{'sense_id': 'küsima_1', 'arg0_feats': None, 'arg1_feats': ['deprel=obj'], 'arg2_feats': ['deprel=obl', 'case=Abl'], 'arg3_feats': None, 'arg4_feats': None, 'arg5_feats': None, 'argm_mnr_feats': None, 'argm_tmp_feats': None, 'argm_loc_feats': None}])])

<div class="alert alert-block alert-warning">
<h4><i>PropBankPreannotator's limitations</i></h4>
The output of PropBankPreannotator can be incomplete and erroneous, as it builds upon work-in-progress resources. 
For commonly used verbs, PropBankPreannotator can output many ambiguous sense interpretations, as no disambiguation has been implemented yet.
<br><br>
</div>